# Features from the paper

`Easley, D., López de Prado, M., O’Hara, M., Zhang, Z., 2020. Microstructure in the Machine Age. Rev. Financ. Stud. 34, 3316–3363`

In [1]:
# import some packages
import numpy as np
import pandas as pd
import os
from pathlib import Path

home = str(Path.home())

In [2]:
# Load the data
data_folder = os.path.join(home, 'data', 'optiver-realized-volatility-prediction')

# for the prototype load just one stock
df_book = pd.read_parquet(os.path.join(data_folder, 'book_train.parquet', 'stock_id=0'))
df_trades = pd.read_parquet(os.path.join(data_folder, 'trade_train.parquet', 'stock_id=0'))
# df_trades

## Features

### 0. Some simple stats

In [3]:
# quoted spread
df_book['spread'] = df_book['ask_price1'] - df_book['bid_price1']


### 1. Roll measure

$R_\tau = 2\sqrt{\text{Cov} (\Delta P_\tau, \Delta P_{\tau-1})}$

In [4]:
df_roll = df_trades
df_roll['d_price'] = df_roll.groupby(['time_id'])['price'].diff()
df_roll['d_price_l1'] = df_roll.groupby(['time_id'])['d_price'].shift(1)

grp = df_roll.groupby('time_id')
df_roll = pd.DataFrame()
df_roll['covariance'] = grp.apply(lambda x: x['d_price'].cov(x['d_price_l1'])) 
df_roll['roll_measure'] = 2 * np.sqrt(np.abs(df_roll.covariance))

df_roll.roll_measure.describe()

# df_roll

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/nanops.py:1516: RuntimeWarning: Degrees of freedom <= 0 for slice
  return np.cov(a, b, ddof=ddof)[0, 1]
/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:2480: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


count    3.826000e+03
mean     3.447380e-04
std      3.805181e-04
min      8.844808e-07
25%      1.475523e-04
50%      2.465249e-04
75%      4.022662e-04
max      7.684689e-03
Name: roll_measure, dtype: float64

## 2. ...